# Import modules


In [149]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#ML auxiliary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

#SkLearn auxiliary libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, top_k_accuracy_score

#SkLearn classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns


In [282]:
len(y_ours)

5203

# Import datasets

In [267]:
our_dataset = pd.read_csv('output_processed.csv')
data = our_dataset.to_numpy()
x_ours = data[:, :-1].astype(float)
y_ours = our_dataset['Family'].values
x_train_ours, x_test_ours, y_train_ours, y_test_ours = train_test_split(x_ours, y_ours, test_size=0.2)
# Assuming x_ours and y_ours are your input features and labels, respectively.
unique_classes = np.unique(np.concatenate([y_train_ours, y_test_ours]))
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
y_train_ours = np.array([class_mapping[cls] for cls in y_train_ours])
y_test_ours = np.array([class_mapping[cls] for cls in y_test_ours])
y_ours = np.array([class_mapping[cls] for cls in y_ours])


# LGBMClassifier

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score

# Assuming x_ours and y_ours are your input features and labels, respectively.

# Split the data into training and testing sets
x_train_ours, x_test_ours, y_train_ours, y_test_ours = train_test_split(x_ours, y_ours, test_size=0.2, random_state=42)

# Create the LightGBM classifier with adjusted parameters
lgbm_model = LGBMClassifier(
    max_bin=500,
    learning_rate=0.05,
    num_iterations=1000,
    num_leaves=50,  # Adjust based on the size of your dataset and complexity
    boosting_type='dart',
    categorical_feature=[0, 1, 2]  # Specify categorical features indices
)

# Use cross-validation to evaluate the model
cv_scores = cross_val_score(lgbm_model, x_ours, y_ours, cv=5, scoring='accuracy')
print("Cross-validation accuracy:", cv_scores)
print("Cross-validation mean accuracy:", cv_scores.mean())


In [287]:
print("Cross-validation accuracy:", cv_scores)
print("Cross-validation mean accuracy:", cv_scores.mean())

Cross-validation accuracy: [1. 1. 1. 1. 1.]
Cross-validation mean accuracy: 1.0


# XGboost

In [284]:


# Assuming x_ours and y_ours are your input features and labels, respectively.

# Split the data into training and testing sets

# Define XGBoost parameters
xgb_params = {
    'colsample_bytree': 0.9,
    'learning_rate': 0.1,
    'max_depth': 9,
    'n_estimators': 200,
    'subsample': 0.9,
    'objective': 'multi:softmax',
    'num_class': len(set(y_train_ours)),
    'eval_metric': 'mlogloss'
}

# Create XGBoost classifier
xgb_model = xgb.XGBClassifier(**xgb_params)

# Use cross-validation to evaluate the model
cv_scores = cross_val_score(xgb_model, x_ours, y_ours, cv=5, scoring='accuracy')
print("Cross-validation accuracy:", cv_scores)

print("Cross-validation mean accuracy:", cv_scores.mean())




Cross-validation accuracy: [1. 1. 1. 1. 1.]
Cross-validation mean accuracy: 1.0


# ANN

In [271]:
y_cat_ours = to_categorical(y_ours)
x_train_ours, x_test_ours, y_train_ours, y_test_ours = train_test_split(x_ours, y_cat_ours, test_size=0.2)

model = Sequential()
model.add(Dense(512, input_dim=70, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(13, activation = "softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', TopKCategoricalAccuracy(k=2)])
model.summary()
model.fit(x_train_ours, y_train_ours, verbose=1, epochs=100, batch_size=10)

predict_x=model.predict(x_test_ours) 
y_pred_class=np.argmax(predict_x,axis=1)

y_pred = model.predict(x_test_ours)
y_test_class = np.argmax(y_test_ours, axis=1)
print(confusion_matrix(y_test_class, y_pred_class))

print(classification_report(y_test_class, y_pred_class))

print('precision: ' + str(precision_score(y_pred_class, y_test_class, average='weighted')))
print('recall: ' + str(recall_score(y_pred_class, y_test_class, average='weighted')))
print('f1-score: ' + str(f1_score(y_pred_class, y_test_class, average='weighted')))
print('accuracy: ' + str(accuracy_score(y_pred_class, y_test_class)))

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_124 (Dense)           (None, 512)               36352     
                                                                 
 dense_125 (Dense)           (None, 256)               131328    
                                                                 
 dense_126 (Dense)           (None, 128)               32896     
                                                                 
 dropout_36 (Dropout)        (None, 128)               0         
                                                                 
 dense_127 (Dense)           (None, 13)                1677      
                                                                 
Total params: 202253 (790.05 KB)
Trainable params: 202253 (790.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
4

## Random Forest

In [270]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, top_k_accuracy_score, confusion_matrix

# Assuming you have x_train_ours, y_train_ours, x_test_ours, y_test_ours

# Create RandomForestClassifier
rand_for_classifier = RandomForestClassifier()

# Perform cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # Adjust the number of splits as needed
cv_scores = cross_val_score(rand_for_classifier, x_ours, y_ours, cv=cv, scoring='accuracy')

# Print cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())



Cross-Validation Scores: [0.97790586 0.97598463 0.97310279 0.97980769 0.97788462]
Mean CV Accuracy: 0.9769371166777507


## K-Nearest Neighbors

In [285]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, top_k_accuracy_score

# Assuming x_ours and y_ours are your full dataset

# Split the data into training and testing sets
x_train_ours, x_test_ours, y_train_ours, y_test_ours = train_test_split(x_ours, y_ours, test_size=0.2, random_state=42)

# Initialize KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

# Perform cross-validation on the training set
cv_scores = cross_val_score(knn, x_train_ours, y_train_ours, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Cross-validation mean score:", cv_scores.mean())

# Fit the model on the training data


Cross-Validation Scores: [0.95438175 0.96398559 0.96033654 0.94711538 0.96033654]
Cross-validation mean score: 0.9572311616954474
